<a href="https://colab.research.google.com/github/yaPhilya/Computer-Vision/blob/master/hw5/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from keras.applications.xception import Xception, preprocess_input

Using TensorFlow backend.


In [0]:
from skimage.transform import resize
from skimage.io import imread
from keras.preprocessing import image

In [0]:
import keras.layers as L
from keras.models import Model

In [0]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/hw5')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/hw5


In [3]:
import os
os.listdir('/content/hw5/My Drive/Colab Notebooks/Computer Vision/hw5')

['images', 'gt.csv', 'imgs.npy']

In [0]:
PATH = '/content/hw5/My Drive/Colab Notebooks/Computer Vision/hw5/'

In [6]:
gt = pd.read_csv(PATH + 'gt.csv')
gt.head()

,filename,class_id
0,0000.jpg,0
1,0001.jpg,0
2,0002.jpg,0
3,0003.jpg,0
4,0004.jpg,0


In [0]:
filenames = []
label = []
for row in gt.iterrows():
  filenames.append(row[1]['filename'])
  label.append(row[1]['class_id'])
label = np.array(label)

In [0]:
NEW_SHAPE = 299

In [0]:

images = []
for file in filenames:
  images.append(image.load_img(PATH + 'images/{}'.format(file), target_size=(NEW_SHAPE, NEW_SHAPE)))
images = np.array(images)

In [0]:
images = np.load(PATH + 'imgs.npy')

In [0]:
processed_imgs = []
for im in images:
  processed_imgs.append(preprocess_input(im))
processed_imgs = np.array(processed_imgs)

In [21]:
l_bin = LabelBinarizer()
target = l_bin.fit_transform(label)
target.shape

(2500, 50)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(processed_imgs, target, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((2000, 299, 299, 3), (500, 299, 299, 3), (2000, 50), (500, 50))

In [13]:
base_model = Xception(weights='imagenet', include_top=False)

83689472/83683744 [==============================] - 7s 0us/step


In [0]:
avg_pool = L.GlobalAveragePooling2D()(base_model.output)
dense = L.Dense(256, activation='relu')(avg_pool)
prediction = L.Dense(label.max() + 1, activation='softmax')(dense)

model = Model(inputs=base_model.input, outputs=prediction)

In [25]:
model.summary(line_length=150)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             (None, None, None, 3)            0                                                                   
______________________________________________________________________________________________________________________________________________________
block1_conv1 (Conv2D)                            (None, None, None, 32)           864               input_1[0][0]                                     
______________________________________________________________________________________________________________________________________________________
block1_conv1_bn (BatchNormalization)             (None, None, None, 32)           128         

In [0]:
for layer in base_model.layers:
  layer.trainable = False

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test))

Train on 2000 samples, validate on 500 samples
Epoch 1/10
2000/2000 [==============================] - 62s 31ms/step - loss: 3.0463 - acc: 0.3065 - val_loss: 2.1976 - val_acc: 0.4780
Epoch 2/10
2000/2000 [==============================] - 45s 22ms/step - loss: 1.4358 - acc: 0.6375 - val_loss: 1.3964 - val_acc: 0.6200
Epoch 3/10
2000/2000 [==============================] - 45s 22ms/step - loss: 0.9122 - acc: 0.7385 - val_loss: 1.1972 - val_acc: 0.6460
Epoch 4/10
2000/2000 [==============================] - 45s 22ms/step - loss: 0.6786 - acc: 0.8065 - val_loss: 1.0720 - val_acc: 0.6620
Epoch 5/10
2000/2000 [==============================] - 45s 22ms/step - loss: 0.5393 - acc: 0.8505 - val_loss: 1.0169 - val_acc: 0.6900
Epoch 6/10
2000/2000 [==============================] - 45s 22ms/step - loss: 0.4459 - acc: 0.8845 - val_loss: 0.9534 - val_acc: 0.6980
Epoch 7/10
2000/2000 [==============================] - 45s 22ms/step - loss: 0.3749 - acc: 0.9075 - val_loss: 0.9220 - val_acc: 0.7040
E

In [24]:
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14 batch_normalization_1 False
15 add_1 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_2 False
23 block3_pool False
24 batch_normalization_2 False
25 add_2 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_3 False
33 block4_pool False
34 batch_normalization_3 False
35 add_3 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2

In [0]:
for layer in model.layers[:126]:
  layer.trainable = False
for layer in model.layers[126:]:
  layer.trainable = True

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_test, y_test))

Train on 2000 samples, validate on 500 samples
Epoch 1/50
2000/2000 [==============================] - 51s 26ms/step - loss: 0.4009 - acc: 0.8720 - val_loss: 0.8352 - val_acc: 0.7400
Epoch 2/50
2000/2000 [==============================] - 48s 24ms/step - loss: 0.1148 - acc: 0.9700 - val_loss: 0.7501 - val_acc: 0.7720
Epoch 3/50
2000/2000 [==============================] - 47s 24ms/step - loss: 0.0407 - acc: 0.9910 - val_loss: 0.8405 - val_acc: 0.7560
Epoch 4/50
2000/2000 [==============================] - 47s 24ms/step - loss: 0.0198 - acc: 0.9945 - val_loss: 0.7249 - val_acc: 0.7820
Epoch 5/50
2000/2000 [==============================] - 48s 24ms/step - loss: 0.0090 - acc: 0.9985 - val_loss: 0.7364 - val_acc: 0.7800
Epoch 6/50
2000/2000 [==============================] - 48s 24ms/step - loss: 0.0043 - acc: 0.9995 - val_loss: 0.7204 - val_acc: 0.7700
Epoch 7/50
2000/2000 [==============================] - 47s 24ms/step - loss: 0.0021 - acc: 0.9995 - val_loss: 0.7032 - val_acc: 0.7940
E

KeyboardInterrupt: ignored

In [0]:
model.save(PATH + 'first_Xception.hdf5')